# DH / DSPI

In [21]:
#!pip install keras
!pip install tensorflow

    100% |████████████████████████████████| 83.1MB 8.3kB/s ta 0:00:011 0% |▎                               | 696kB 3.2MB/s eta 0:00:26    1% |▎                               | 839kB 2.4MB/s eta 0:00:35    13% |████▏                           | 10.9MB 3.1MB/s eta 0:00:24    16% |█████▎                          | 13.6MB 2.9MB/s eta 0:00:25    16% |█████▎                          | 13.8MB 4.5MB/s eta 0:00:16    17% |█████▌                          | 14.3MB 2.5MB/s eta 0:00:28    20% |██████▍                         | 16.6MB 5.5MB/s eta 0:00:13    22% |███████                         | 18.4MB 3.3MB/s eta 0:00:20    24% |████████                        | 20.5MB 3.0MB/s eta 0:00:22    26% |████████▍                       | 21.9MB 4.4MB/s eta 0:00:14    28% |█████████                       | 23.5MB 5.2MB/s eta 0:00:12    28% |█████████▏                      | 23.9MB 3.3MB/s eta 0:00:19    29% |█████████▍                      | 24.4MB 5.9MB/s eta 0:00:10    30% |█████████▊                     

    100% |████████████████████████████████| 102kB 2.4MB/s a 0:00:01    87% |███████████████████████████▉    | 81kB 5.6MB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 1.4MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 1.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/18/ea/5e/e36e1b8739e78cd2eba0a08fdc602c2b16a4b263912af8cb64
Successfully built termcolor gast absl-py
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
      Successfully uninstalled numpy-1.12.1
  Found existing installation: protobuf 3.5.1
    Uninstalling protob

In [22]:
import csv
import re
#from io import StringIO
import pandas as pd
import numpy as np

import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error,matthews_corrcoef,classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation
from keras.layers.advanced_activations import PReLU
from keras.callbacks import CSVLogger,EarlyStopping, ModelCheckpoint



Using TensorFlow backend.


In [108]:
#Funciones
#Separa columna tomada de logs en campos
def splitdata(col):
    temp = []
    for i in col:
        t = (i.split('='))
        temp.append(t[0])           
    return temp

#Normmaliza Dataset:
def splitdf(df):
    df = df.ix[:, ~df.columns.str.contains('Unnamed')]
    #r = re.compile(r"[^a-zA-Z0-9-]")
    df2 = pd.DataFrame(columns=df.columns)
    for i in df:
        new = df[i].str.split('=',n=1, expand=True)
        df2[i] = new[1]#.str.extract(r'([a-zA-Z0-9=])')
        #df2[i] = r.sub("",new[1])
    return df2

def fortidf(df):
    lcol = [] 
    lcol = df.columns
    df.columns = splitdata(lcol)
    df2 = splitdf(df)
    df2['ioc'] = 0
    return df2

def iptoint(ip):
    tmp = re.findall(r"\d+", ip)
    iplist = "".join(map(str, tmp))
    return iplist

In [60]:
#Importa CSV a DF
#df = Logs identificados como maliciosos
#df2 = Logs de trafico
df2 = pd.read_csv('Traffic.csv')
df = pd.read_csv('botnet.csv')
df = fortidf(df)
df2 = fortidf(df2)

#[^=]*

In [61]:
#Columnas comunes entre logs de Trafico y logs de Botnet
scolumns=['date', 'time', 'devid', 'vd', 'type', 'subtype', 'action',
       'app', 'appcat','craction', 'crscore', 'devname',
       'dstcountry', 'dstintf', 'dstip', 'dstport', 'duration', 'dvid','level','logid', 'logver', 'policyid','proto', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'threatcnts', 'threatlvls', 'threats',
       'threattyps', 'threatwgts', 'trandisp', 'tranip', 'tranport', 'ioc']

In [64]:
#Join de datos
data = pd.concat([df,df2])
data.sample(3)

,action,app,appact,appcat,appid,applist,apprisk,apps,bid,countapp,...,time,trandisp,tranip,tranport,transip,transport,type,utmaction,utmref,vd
10,close,HTTPS,NaN,unscanned,NaN,NaN,NaN,NaN,44007401,NaN,...,12:21:59,dnat,192.168.14.13,443,NaN,NaN,traffic,NaN,NaN,REF131396
40,accept,DNS,NaN,Network.Service,NaN,NaN,NaN,NaN,44007393,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1008343
3,close,Conficker.Botnet,drop-session,Botnet,17201,block-p2p,critical,{Conficker.Botnet},100106760,1,...,12:36:49,snat,NaN,NaN,200.41.178.10,62110,traffic,block,BAQAAAAIAAAB3EAGyVQFY4B8L_ANcchABsFUBWOAfC_wDXA==,root


In [69]:
#Seleccion de features
features = ['itime','srcip','dstip','dstport','duration', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt','service','ioc' ]
data.loc[:, features].sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,service,ioc
10,1544368919,138.118.214.38,200.41.197.1,443,10,88,2,132,3,HTTPS,0
1,1544368919,45.237.106.242,200.41.197.1,443,14,112,2,244,5,HTTPS,0
6,1544368919,190.16.91.39,200.41.197.1,443,10,88,2,132,3,HTTPS,0


In [136]:
#Clasifica infectados:
data.loc[data.appcat == 'Botnet', 'ioc'] = 1
data.loc[data.rcvdpkt == 'None', 'rcvdpkt'] = 0
data.loc[data.sentpkt == 'None', 'sentpkt'] = 0



#rcvdpkt

In [165]:
data2 = data.loc[:, features]
data2.sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,service,ioc
1,1543765017,176.16.1.5,104.244.14.252,80,6,3039,4,128,3,HTTP,1
80,1544368919,192.168.101.83,131.107.13.100,123,60,0,None,76,1,NTP,0
40,1544368919,10.221.61.232,8.8.4.4,53,60,76,1,60,1,DNS,0


In [166]:
#Convierte IP en int
data2.srcip = data2.srcip.apply(iptoint)
data2.dstip = data2.dstip.apply(iptoint)


In [167]:
#Convierte servicios en dummy
data2 = pd.get_dummies(data2, columns=['service'])
data2.sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,ioc,service_443/udp,service_8183/tcp,service_DNS,service_HTTP,service_HTTPS,service_NTP,service_SMTP
10,1544368919,13811821438,200411971,443,10,88,2,132,3,0,0,0,0,0,1,0,0
22,1544368919,9344105232,200411971,443,5,60,1,100,2,0,0,0,0,0,1,0,0
41,1544368919,1022117213,31139419,443,191,578,6,336,7,0,0,0,0,0,1,0,0


In [168]:
#Reemplaza NaN por 0
data2.fillna(0)
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 98
Data columns (total 17 columns):
itime               113 non-null object
srcip               113 non-null object
dstip               113 non-null object
dstport             113 non-null object
duration            113 non-null object
rcvdbyte            113 non-null object
rcvdpkt             101 non-null object
sentbyte            113 non-null object
sentpkt             104 non-null object
ioc                 113 non-null int64
service_443/udp     113 non-null uint8
service_8183/tcp    113 non-null uint8
service_DNS         113 non-null uint8
service_HTTP        113 non-null uint8
service_HTTPS       113 non-null uint8
service_NTP         113 non-null uint8
service_SMTP        113 non-null uint8
dtypes: int64(1), object(9), uint8(7)
memory usage: 10.5+ KB


In [180]:
X = data2
y = data2.ioc
X.drop(['ioc'], inplace=True, axis=1)

0    1
1    1
2    1
3    1
4    1
Name: ioc, dtype: int64

In [132]:
print(train.shape)
print(test.shape)

(79, 17)
(34, 17)


In [133]:
train_X = train.iloc[:, 1:31

SyntaxError: unexpected EOF while parsing (<ipython-input-133-a6d00a20d8de>, line 1)

In [134]:
train

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,ioc,service_443/udp,service_8183/tcp,service_DNS,service_HTTP,service_HTTPS,service_NTP,service_SMTP
6,1544368919,190169139,200411971,443,10,88,2,132,3,0,0,0,0,0,1,0,0
90,1544368919,1921683057,94100180150,25,8,128,3,210,4,0,0,0,0,0,0,0,1
77,1544368919,19216810176,16212518133,443,188,132,3,212,5,0,0,0,0,0,1,0,0
11,1544368919,19216819711,20041235242,443,1,92,2,132,3,0,0,0,0,0,1,0,0
40,1544368919,1022161232,8844,53,60,76,1,60,1,0,0,0,1,0,0,0,0
4,1543765009,1761615,10424414252,80,6,3039,4,128,3,1,0,0,0,1,0,0,0
58,1544368919,101204857,31139435,443,0,0,None,0,None,0,0,0,0,0,1,0,0
10,1543764987,1761615,10424414252,80,6,3039,4,128,3,1,0,0,0,1,0,0,0
37,1544368919,102215223,17221730229,443,245,722,8,1636,5,0,0,0,0,0,1,0,0
92,1544368919,19216850227,17221716214,443,31,216,4,399,4,0,0,0,0,0,1,0,0


In [102]:
model= DecisionTreeClassifier(random_state=1234)
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print("Accuracy:",metrics.accuracy_score(prediction,test_y))



NameError: name 'train_y' is not defined